In [1]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from plotnine import *
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from dla_utils import _dla_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We re

In [2]:
alt.themes.register("calitp_theme", styleguide.calitp_theme)
# enable
alt.themes.enable("calitp_theme")

ThemeRegistry.enable('calitp_theme')

In [3]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Lato');
</style>


In [4]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/dla_df.parquet")

In [5]:
df.sample(1)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
20114,Obligated,ATPL,5208(159),Clovis,2021-10-04,2021-10-04,2021-10-04,2021-10-04,2021-10-15,-3.86,...,0.00,Large,1,0,0,0,0,0,0,1


## Function to Count by Year

In [6]:
df_years = _dla_utils.count_all_years((df>>filter(_.dist==7)))

In [7]:
#from Tiffany's branch DLA functions
def count_all_years(df, groupedby=["dist"]):
    count_years = geography_utils.aggregate_by_geography(
        df, 
        group_cols = ["prepared_y", "dist"],
        sum_cols = ["total_requested", "ac_requested", "fed_requested"],
        mean_cols = ["total_requested", "ac_requested", "fed_requested"],
        nunique_cols = ["primary_agency_name", "prefix", "project_no", "project_location", "type_of_work"]
    ).sort_values(["prepared_y", "dist"], ascending=[False, True])
    
    count_years= count_years.rename(columns={"ac_requested_x": "ac_requested_sum",
                                "fed_requested_x": "fed_requested_sum",
                                "total_requested_x": "total_requested_sum",
                                "ac_requested_y": "ac_requested_mean",
                                "fed_requested_y": "fed_requested_mean",
                                "total_requested_y": "total_requested_mean",
                                "status": "counts",
                                "prefix": "unique_prefix",
                                "primary_agency_name": "unique_primary_agency_name",
                                "project_location": "unique_project_location",
                                "project_no":"unique_project_no",
                                "type_of_work":"unique_type_of_work"})

    return count_years

In [8]:
df1 = count_all_years(df)

In [9]:
df1

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
120,"2,022.00",1,"-3,038,306.36","12,136,866.49","10,429,434.12","-34,526.21","137,918.94","118,516.30",9.00,6.00,83.00,83.00,29.00
121,"2,022.00",2,"-1,305,386.31","3,219,171.96","2,046,330.61","-43,512.88","107,305.73","68,211.02",9.00,11.00,29.00,29.00,25.00
122,"2,022.00",3,"-5,317,762.02","83,539,755.73","104,642,173.73","-51,132.33","803,266.88","1,006,174.75",26.00,28.00,92.00,92.00,75.00
123,"2,022.00",4,"-19,637,178.90","63,931,553.61","55,685,262.48","-160,960.48","524,029.13","456,436.58",21.00,58.00,106.00,106.00,99.00
124,"2,022.00",5,"-24,197,970.42","23,001,143.54","-1,122,576.85","-483,959.41","460,022.87","-22,451.54",8.00,13.00,46.00,46.00,44.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df2 = count_all_years((df>>filter(_.dist==4)))

In [11]:
df2

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
12,"2,022.00",4,"-19,637,178.90","63,931,553.61","55,685,262.48","-160,960.48","524,029.13","456,436.58",21.00,58.00,106.00,106.00,99.00
10,"2,021.00",4,"-7,581,867.92","150,810,954.28","207,275,549.66","-18,537.57","368,730.94","506,786.18",42.00,102.00,319.00,329.00,292.00
9,"2,020.00",4,"-16,015,853.21","215,076,760.46","380,929,514.28","-55,999.49","752,016.64","1,331,921.38",34.00,89.00,234.00,243.00,215.00
8,"2,019.00",4,"-70,929,402.87","314,851,234.42","319,675,519.69","-177,323.51","787,128.09","799,188.80",41.00,93.00,322.00,333.00,299.00
0,"2,018.00",4,"-19,612,828.51","286,153,969.05","374,039,033.36","-43,200.06","630,295.09","823,874.52",51.00,97.00,344.00,369.00,315.00
1,"2,017.00",4,"-12,268,568.69","279,804,721.77","332,810,620.13","-33,429.34","762,410.69","906,840.93",43.00,97.00,292.00,319.00,269.00
2,"2,016.00",4,"59,687,248.94","304,782,072.96","502,320,621.82","144,172.10","736,188.58","1,213,334.84",50.00,113.00,336.00,361.00,300.00
3,"2,015.00",4,"68,364,142.00","239,952,043.43","448,655,372.86","152,258.67","534,414.35","999,232.46",48.00,109.00,374.00,394.00,300.00
4,"2,014.00",4,"-21,667,366.00","178,332,621.75","229,278,385.00","-44,491.51","366,186.08","470,797.51",52.00,108.00,376.00,400.00,295.00
5,"2,013.00",4,"928,825.00","48,608,230.09","57,966,808.58","24,442.76","1,279,163.95","1,525,442.33",15.00,28.00,35.00,38.00,38.00


In [12]:
df3= count_all_years((df>>filter(_.prefix=="CML")))

In [13]:
df3

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
90,"2,022.00",3,"2,840,642.00","2,103,134.31","6,560,517.31","710,160.50","525,783.58","1,640,129.33",1,4,4,4,4
91,"2,022.00",4,0.00,"6,470,029.09","9,193,648.33",0.00,"647,002.91","919,364.83",1,7,10,10,10
92,"2,022.00",6,0.00,"10,629,878.05","15,576,522.33",0.00,"379,638.50","556,304.37",1,13,26,26,25
93,"2,022.00",7,0.00,"2,445,000.00","1,282,505.01",0.00,"244,500.00","128,250.50",1,6,10,10,10
95,"2,022.00",8,0.00,"-405,164.88",0.00,0.00,"-101,291.22",0.00,1,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"2,013.00",10,0.00,"-19,261.00","-191,630.00",0.00,"-9,630.50","-95,815.00",1,2,2,2,2
55,"2,013.00",11,0.00,"821,000.00","927,648.00",0.00,"273,666.67","309,216.00",1,3,3,3,3
58,"2,013.00",12,0.00,"546,814.00","620,993.00",0.00,"182,271.33","206,997.67",1,1,3,3,1
49,"2,012.00",4,0.00,"3,842,690.66","10,576,983.00",0.00,"1,280,896.89","3,525,661.00",1,2,3,3,3


In [14]:
count_all_years(df>>filter(_.primary_agency_name=="Humboldt County"))

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
9,"2,022.00",1,"-2,040,454.75","10,524,514.30","9,807,289.53","-27,206.06","140,326.86","130,763.86",7,1,70,70,20
8,"2,021.00",1,"4,776,055.24","4,824,894.61","10,617,008.32","67,268.38","67,956.26","149,535.33",9,1,63,62,32
7,"2,020.00",1,"-3,136,109.97","12,204,768.40","10,262,741.19","-37,334.64","145,294.86","122,175.49",9,1,59,60,32
6,"2,019.00",1,"4,946,140.05","14,222,150.75","21,295,247.46","33,647.21","96,749.32","144,865.63",9,1,77,78,43
0,"2,018.00",1,"177,241.53","3,015,726.53","3,600,444.63","1,036.50","17,635.83","21,055.23",8,1,110,111,37
1,"2,017.00",1,0.00,"5,158,080.59","5,531,542.79",0.00,"184,217.16","197,555.10",8,1,24,24,16
3,"2,016.00",1,0.00,"4,649,799.00","5,028,074.27",0.00,"140,903.00","152,365.89",10,1,29,30,21
2,"2,015.00",1,0.00,"4,659,366.97","5,363,881.78",0.00,"89,603.21","103,151.57",8,1,35,35,22
4,"2,014.00",1,0.00,"4,554,254.42","4,981,763.78",0.00,"89,299.11","97,681.64",9,1,45,45,23
5,"2,013.00",1,0.00,"100,863.00","113,931.00",0.00,"100,863.00","113,931.00",1,1,1,1,1


## Chart Part 1: Aggregate the Data

In [15]:
def part1_chart(df, col, aggregate_by=["dist"]):
    aggfunc = (
        geography_utils.aggregate_by_geography(
            df[df[aggregate_by].notna()], 
            group_cols = [aggregate_by],
            nunique_cols = [col]))       
    return aggfunc

In [16]:
part1_chart(df, "type_of_work", aggregate_by="primary_agency_name")

,primary_agency_name,type_of_work
0,Humboldt County,90
1,Mendocino County,54
2,Sacramento County,84
3,Sutter County,12
4,Contra Costa County,60
...,...,...
614,Palos Verdes Est,1
615,San Diego Metropolitan Transit System,1
616,Corte Madera,1
617,Hillsborough,1


## Find Top Nuniques

In [17]:
## want to create a function that gets a new dataframe that will hold all the counts and top twenty 
def find_top(df):
    
    cols= [["prefix",
           "prepared_y",
           "status_comment",
           "project_location",
           "type_of_work",
           "seq",
           "mpo",
           "primary_agency_name"]]
    
    tops_1 = (df
          >>count(_.prefix)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_prefix"})
    tops_2 = (df
          >>count(_.prepared_y)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_prepared_y"})
    tops_3 = (df
          >>count(_.status_comment)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_status_comment"})
    tops_4 = (df
          >>count(_.project_location)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_project_location"})
    tops_5 = (df
          >>count(_.type_of_work)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_type_of_work"})
    tops_6 = (df
          >>count(_.mpo)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_mpo"})
    tops_7 = (df
          >>count(_.primary_agency_name)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_primary_agency_name"})
    
    
    test = tops_1.join(tops_2, how='outer')
    test2 = test.join(tops_3, how='outer')
    test3 = test2.join(tops_4, how='outer')
    test4 = test3.join(tops_5, how='outer')
    test5 = test4.join(tops_6, how='outer')
    tops = test5.join(tops_7, how='outer')
                       
    return tops

In [18]:
tops = find_top(df)
tops

,prefix,unique_prefix,prepared_y,unique_prepared_y,status_comment,unique_status_comment,project_location,unique_project_location,type_of_work,unique_type_of_work,mpo,unique_mpo,primary_agency_name,unique_primary_agency_name
0,HSIPL,3463,"2,018.00","2,819.00",Authorized,20498,San Francisco Bay Area,68,Bridge Replacement (tc),727,SCAG,5235,Humboldt County,713
1,CML,3441,"2,014.00","2,724.00",..,190,Various Locations,32,Bridge Replacement,553,MTC,3419,Los Angeles,508
2,STPL,2909,"2,019.00","2,655.00",Prog Code Z400,108,Sacog Region,29,FTA Transfer,471,NON-MPO,2766,Los Angeles County,488
3,ER,1800,"2,015.00","2,575.00",Prog Code M400,76,Golden Gate Bridge,27,Permanent Restoration,324,SACOG,1946,Santa Cruz County,482
4,BRLO,1496,"2,016.00","2,503.00",Prog Code Z230,47,Throughout Kern County,24,Road Rehabilitation,273,CFCG,1455,Fresno,471
5,BRLS,957,"2,017.00","2,329.00",Prog Code Z003,30,Citywide,18,Bridge Preventive Maintenance,178,KCOG,1045,El Dorado County,312
6,ATPL,627,"2,020.00","2,326.00",Prog Code M40E,26,FTA transfer,16,Emergency Opening,162,AMBAG,797,Kern County,301
7,RPSTPL,591,"2,021.00","2,317.00",Prog Code M0E3,22,Various Locations (see Comments),14,Storm Damage Restoration,129,STANCOG,795,Bakersfield,280
8,HPLUL,344,"2,022.00",740.00,Prog Code M23E,17,Various Locations In San Diego County,14,Pavement Rehabilitation,125,SJCG,553,Fresno County,280
9,SRTSL,332,"2,013.00",193.00,Prog Code Z301,14,County Of Madera,13,Seismic Retrofit,123,SDAG,518,Stockton,277


In [19]:
_dla_utils.basic_bar_chart(tops, "prepared_y","unique_prepared_y", "prepared_y", "subset")

alt.Chart(...)

In [20]:
df2.sample()

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
6,"2,012.00",4,0.00,"3,652,066.60","10,376,579.05",0.00,"913,016.65","2,594,144.76",2.00,3.00,4.00,4.00,4.00


In [21]:
_dla_utils.basic_bar_chart(
    (df2), "prepared_y", "unique_prefix", "prepared_y", "subset", ""
)

alt.Chart(...)

In [22]:
_dla_utils.basic_bar_chart(
    (tops),"unique_primary_agency_name", "primary_agency_name", "primary_agency_name", "subset", ""
)

alt.Chart(...)

In [23]:
_dla_utils.basic_scatter_chart(df1, "unique_primary_agency_name", "total_requested_sum", "dist", "subset", "")

alt.Chart(...)

In [32]:
_dla_utils.basic_line_chart(df1, "prepared_y:O", "total_requested_sum", "subset", "")

alt.Chart(...)

In [30]:
_dla_utils.basic_line_chart(df1, "prepared_y", "ac_requested_sum", "subset", "")

alt.Chart(...)

#### Issues/what to look for:

* get a df of information not grouped by year with 
    - total requested
    - ac requested 
    - fed requested
* fix name for charts of find_top 


## Chart Function

### Old Function with filtering in the function

In [39]:

def basic_agg(df, col, aggregate_by=["dist"]):
    df1 = ((df >> group_by(_[aggregate_by]) >> summarize(n=_[col].nunique()) >> arrange(-_.n)).head(20))
    chart = (alt.Chart(df1)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=f"{_dla_utils.labeling(aggregate_by)}"),
                 y=alt.Y("n", title=f"Number of Unique {_dla_utils.labeling(col)}"),
                 color = alt.Color("n",
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=f"{(_dla_utils.labeling(col))}")
                                  ))
             .properties( 
                          title=f"Number of Unique {_dla_utils.labeling(col)} by {_dla_utils.labeling(aggregate_by)}")
    )

    return chart

In [40]:
basic_agg(df, "prefix", aggregate_by="primary_agency_name")

alt.Chart(...)

## Function for Selecting Calculating Method

In [41]:
def calculate_data(df, col, aggregate_by=["dist"], aggfunc="sum"):
    df1 = (df.groupby(aggregate_by)
          .agg({col: aggfunc})
          .reset_index()
          .sort_values(col, ascending=False)
         )
    #df = df[df[col] > 0]
    df1 = (df1>>arrange(-_[col])).head(20)
    return df1

In [42]:
calculate_data(df, "total_requested", aggregate_by="dist", aggfunc="sum")

,dist,total_requested
7,7,"4,511,609,279.52"
8,8,"3,347,009,098.99"
4,4,"2,922,037,710.95"
12,12,"2,870,621,628.36"
3,3,"1,663,449,026.64"
6,6,"1,646,031,873.89"
11,11,"1,154,607,072.07"
10,10,"1,085,645,851.34"
5,5,"508,086,469.78"
2,2,"233,866,650.73"


In [43]:
# #calcualte_sum2 might be more versatile
# def calculate_sum(df, col, agg_col):
#     dftest = geography_utils.aggregate_by_geography(
#         df, 
#         group_cols = [agg_col],
#         sum_cols = [col])
#     dftest = (dftest>>arrange(-_[col])).head(20)
#     #dftest = (dftest>>filter(_[col] >0))
#     return dftest


In [44]:
# calculate_sum(df, "total_requested", agg_col="dist")

## Adding to Charting Function

In [45]:
def test_chartfunc_sum(df, col, aggregate_by, aggfunc):
    
    df1 = (calculate_data(df, col, aggregate_by, aggfunc))
    
    chart = (alt.Chart(df1)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=_dla_utils.labeling(aggregate_by), sort=('-y')),
                 y=alt.Y(col, title=_dla_utils.labeling(col)),
                 color = alt.Color(col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(_dla_utils.labeling(col)))
                                  ))
             .properties( 
                          title=f"Highest {_dla_utils.labeling(col)} {_dla_utils.labeling(aggfunc)}s by {_dla_utils.labeling(aggregate_by)}")
    )
    
    return chart



In [46]:
test_chartfunc_sum(df, "total_requested", aggregate_by="type_of_work", aggfunc="mean")

alt.Chart(...)

In [47]:
test_chartfunc_sum(df, "prefix", aggregate_by="primary_agency_name", aggfunc="nunique")

alt.Chart(...)

In [48]:
#making sure it is right
df>>group_by(_.primary_agency_name)>> summarize(n=_.prefix.nunique()) >> arrange(-_.n)

,primary_agency_name,n
276,Los Angeles,35
277,Los Angeles County,31
434,San Bernardino Associated Governments,28
71,Caltrans,24
534,Stockton,23
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


## Grouping Charts

In [49]:
#groupby_col_x axis = prepared_y
#agg_by_col = dist or mpo 
#sum_col= primary_agency_name or prefix
 
def group_chart_nunique(df, groupby_col_x, agg_by_col, sum_col):
    dist_years1 = (df >> group_by(_[groupby_col_x], _[agg_by_col]) 
                   >> summarize(n=_[sum_col].nunique()) 
                   >> arrange(-_[groupby_col_x]))

    chart = alt.Chart(dist_years1).mark_bar().encode(
            column=(f"{agg_by_col}:N"),
            x=alt.X((f"{groupby_col_x}:O"), title=_dla_utils.labeling(groupby_col_x)),
            y=alt.Y('n:Q', title=(f"Number of Unique {_dla_utils.labeling(sum_col)}")),
            color = alt.Color((f"{agg_by_col}:N"), 
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),  
                                   legend=alt.Legend(title=(_dla_utils.labeling(sum_col)))
                                  )
                                  )
    #chart.save(f"./chart_outputs/grouped_{agg_by_col}_by_{groupby_col_x}.png")
    
    return chart


In [50]:
def group_chart_nunique_2(df, groupby_col_x, agg_by_col, sum_col):
    #without function
    # dist_years1 = (df >> group_by(_[groupby_col_x], _[agg_by_col]) 
    #                >> summarize(n=_[sum_col].nunique()) 
    #                >> arrange(-_[groupby_col_x]))

    chart = alt.Chart(df).mark_bar().encode(
            column=(f"{agg_by_col}:N"),
            x=alt.X((f"{groupby_col_x}:O"), title=_dla_utils.labeling(groupby_col_x)),
            y=alt.Y('n:Q', title=(f"Number of Unique {_dla_utils.labeling(sum_col)}")),
            color = alt.Color((f"{agg_by_col}:N"), 
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),  
                                   legend=alt.Legend(title=(_dla_utils.labeling(sum_col)))
                                  )
                                  )
   # chart.save(f"./chart_outputs/grouped_{agg_by_col}_by_{groupby_col_x}.png")
    
    return chart

In [51]:
dist_years1 = (df >> group_by(_.prepared_y, _.mpo)
               >> summarize(n=_.prefix.nunique()) 
               >> arrange(-_.prepared_y))

In [52]:
dist_years1

,prepared_y,mpo,n
195,"2,022.00",AMBAG,6
196,"2,022.00",BCAG,7
197,"2,022.00",CFCG,10
198,"2,022.00",KCAG,3
199,"2,022.00",KCOG,11
...,...,...,...
1,"2,011.00",MTC,1
2,"2,011.00",NON-MPO,1
3,"2,011.00",SCAG,2
4,"2,011.00",STANCOG,1


In [53]:
df2

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
12,"2,022.00",4,"-19,637,178.90","63,931,553.61","55,685,262.48","-160,960.48","524,029.13","456,436.58",21.00,58.00,106.00,106.00,99.00
10,"2,021.00",4,"-7,581,867.92","150,810,954.28","207,275,549.66","-18,537.57","368,730.94","506,786.18",42.00,102.00,319.00,329.00,292.00
9,"2,020.00",4,"-16,015,853.21","215,076,760.46","380,929,514.28","-55,999.49","752,016.64","1,331,921.38",34.00,89.00,234.00,243.00,215.00
8,"2,019.00",4,"-70,929,402.87","314,851,234.42","319,675,519.69","-177,323.51","787,128.09","799,188.80",41.00,93.00,322.00,333.00,299.00
0,"2,018.00",4,"-19,612,828.51","286,153,969.05","374,039,033.36","-43,200.06","630,295.09","823,874.52",51.00,97.00,344.00,369.00,315.00
1,"2,017.00",4,"-12,268,568.69","279,804,721.77","332,810,620.13","-33,429.34","762,410.69","906,840.93",43.00,97.00,292.00,319.00,269.00
2,"2,016.00",4,"59,687,248.94","304,782,072.96","502,320,621.82","144,172.10","736,188.58","1,213,334.84",50.00,113.00,336.00,361.00,300.00
3,"2,015.00",4,"68,364,142.00","239,952,043.43","448,655,372.86","152,258.67","534,414.35","999,232.46",48.00,109.00,374.00,394.00,300.00
4,"2,014.00",4,"-21,667,366.00","178,332,621.75","229,278,385.00","-44,491.51","366,186.08","470,797.51",52.00,108.00,376.00,400.00,295.00
5,"2,013.00",4,"928,825.00","48,608,230.09","57,966,808.58","24,442.76","1,279,163.95","1,525,442.33",15.00,28.00,35.00,38.00,38.00


## Plot Nine

In [54]:
def prefix_all_agencies(df, prefix_unique):
    
    # graphs 
    prefixes = df[df.prefix== prefix_unique]
    
    prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(50)
    
    prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix_unique} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix_unique)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
                + theme_538()
                + theme(plot_background=element_rect(fill=backgroundColor, color=backgroundColor),
                     panel_background=element_rect(fill=backgroundColor, color=backgroundColor),
                     panel_grid_major_y=element_line(
                        color=axisColor, linetype='solid', size=1),
                     panel_grid_major_x=element_blank(),
                     figure_size=(7.0, 4.4),
                     title=element_text(weight="bold", size=font_size, 
                                        family=font, color=blackTitle),
                     axis_title=element_text(family=labelFont, size=12, color=guideTitleColor),
                     axis_text=element_text(family=labelFont, size=10, color=guideLabelColor, 
                                            margin={'r': 4}
                                           ),
                     axis_title_x=element_text(margin={'t': 10}),
                     axis_title_y=element_text(margin={'r': 10}),
                     legend_title=element_text(font=labelFont, size=14, color=blackTitle, 
                                               margin={'b': 10}),
                     legend_text=element_text(font=labelFont, size=11, color=blackTitle, 
                                              margin={'t': 5, 'b': 5, 'r': 5, 'l': 5}),
                    )
            )    
    #ax1 = preset_plotnine_config(ax1)
    return ax1
  

In [55]:
prefixes = df[df.prefix== "CML"]
    
prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(50)
    
prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)

In [56]:
from shared_utils import calitp_color_palette as cp

In [57]:
# ax1 = (prefix_count
#             >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
#                 + geom_col() 
#                 + theme(axis_text_x = element_text(angle = 45 , hjust=1))
#                 + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
#             )

# chart = (styleguide.preset_plotnine_config(ax1)
#          + scale_fill_manual(values=cp.CALITP_SEQUENTIAL_COLORS)
#         )

# chart

# Misc.

In [58]:
# def totalfunds_chart_(df, aggregate_by=[], col):
#      aggfunc = (
#         geography_utils.aggregate_by_geography(
#             df[df[aggregate_by].notna()], 
#             group_cols = [aggregate_by],
#             sum_cols = [col],
#             nunique_cols = [col]

            
    
#     test= (df
#            >>group_by(_.primary_agency_name)
#            >>summarize(Total_Funds = _[col].sum())
#            >>arrange(-_.Total_Funds))
#     #test.rename(columns={'primary_agency_name': 'Primary Agency Name', 'col': 'Sum Funds'}, inplace=True)
#     test.style.format(precision=2, na_rep='MISSING', thousands=",")
#     chart = (alt.Chart(test.head(20))
#              .mark_bar()
#              .encode(
#                  x=alt.X("primary_agency_name", title="col"),
#                  y=alt.Y("Total Funds", title="Total Funds Obligated"),
#                  #column = "payment:N",
#                  color = alt.Color("primary_agency_name", 
#                                   scale=alt.Scale(
#                                       range=altair_utils.FIVETHIRTYEIGHT_DIVERGING_COLORS
#                                   )
#                                   )
#              )
#             .properties( 
#                            title="District 7 Agencies with the Most Total Funds Obligated")

#     )
#     return chart